## Load the Needed Packages

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydrant.topology.geom as gm
import subprocess
import os
from   shapely.geometry import Point
import yaml
import shutil
import warnings
import networkx as nx
import re
import copy
from   HydroLakes_MERITBasins_HydroLakesCorrection import FixHydroLakesForMerit
from   GeneralFunctions import add_immediate_upstream, create_graph, count_network_parts
warnings.filterwarnings("ignore")

# Get the Configuration Information for Files and Folders

In [2]:
# Load general configurations from a YAML file
yaml_file = os.path.abspath('./HydroLakes_MERITBasins_Config.yaml')
with open(yaml_file, 'r') as f:
    GeneralConfig = yaml.safe_load(f)

# get the path to the merit-basins files
path_out             = GeneralConfig['DomainFolder']
TempFolder           = GeneralConfig['TempFolder']
pfafs                = GeneralConfig['pfafs']
lake_file            = GeneralConfig['LakeFile']
IncludeLakes         = GeneralConfig['IncludeLakes']
riv_path             = GeneralConfig['MERITBasinsPaths']['RivPath']
cat_path             = GeneralConfig['MERITBasinsPaths']['CatPath']
cst_path             = GeneralConfig['MERITBasinsPaths']['CstPath']
riv_file_template    = GeneralConfig['MERITBasinsPaths']['RivFileTemplate']
cat_file_template    = GeneralConfig['MERITBasinsPaths']['CatFileTemplate']
cst_file_template    = GeneralConfig['MERITBasinsPaths']['CstFileTemplate']

# Prepare each PFAF river and subbasins (including costal hillslope) for burning the lakes

In [3]:
if IncludeLakes:
    # load hydrolakeDataset
    lakeO = gpd.read_file(lake_file) # read the hydrolake dataset
    lakeO = FixHydroLakesForMerit(lakeO) # correct for hydrolakes for merit
    lakeO['x'] = lakeO.centroid.x # add centroid lon
    lakeO['y'] = lakeO.centroid.y # add centroid lat

# loop over the config files:
for pfaf in pfafs:

    # get the list of pfaf for the subdomain
    subdomain = 'pfaf'+pfaf
    
    # create the folder and domain to include
    path_out_temp = TempFolder+'pfaf'+pfaf+'/'+subdomain+'/'
    if os.path.isdir(path_out_temp):
        try:
            shutil.rmtree(path_out_temp)  # Remove the entire directory and its contents
        except OSError as e:
            print(f"Error: {path_out_temp} - {e}")
    if not os.path.isdir(path_out_temp):
        os.makedirs(path_out_temp)

    # read in the files
    riv, cat = gm.merit_read_file ([pfaf],
                                   riv_path,
                                   riv_file_template,
                                   cat_path,
                                   cat_file_template,
                                   path_cst = cst_path,
                                   cst_file_template = cst_file_template)
    
    # prepare the ntopo for merit
    riv, cat = gm.prepare_ntopo(riv=riv,\
                                riv_cols={'id':'COMID', 'next_id':'NextDownID'},\
                                cat=cat,\
                                cat_cols={'id':'COMID', 'hillslope':'hillslope', 'area' :'unitarea'},\
                                network = 'merit')

    # make sure the 0 or negative values are set to -9999 as NextDownID
    riv['NextDownID'] = riv['NextDownID'].mask(riv['NextDownID'] <= 0, -9999)
    riv.loc[riv['hillslope'] == 1, ['lengthkm']] = 0 # set the length to 0 for coastal hillslope

    # make sure both riv and cat have the same COMID order and values sorted
    # Sort both dataframes to identical COMID order
    riv = riv.sort_values("COMID").reset_index(drop=True)
    cat = cat.sort_values("COMID").reset_index(drop=True)

    # check if the COMID are exactly similar in two different values
    riv_ids = set(riv["COMID"])
    cat_ids = set(cat["COMID"])
    if riv_ids != cat_ids:
        missing_in_riv = cat_ids - riv_ids
        missing_in_cat = riv_ids - cat_ids
        raise ValueError(
            f"COMID mismatch:\n"
            f"  Missing in riv: {missing_in_riv}\n"
            f"  Missing in cat: {missing_in_cat}")

    # make sure the fields of COMID, NextDownCOMID, unitarea, uparea, length
    # and length are defined in both riv
    riv ['length'] = riv['lengthkm']
    riv ['NextDownCOMID'] = riv['NextDownID']
    cols_to_keep = ["COMID", "NextDownCOMID", "length", "unitarea", "uparea", "geometry"]
    missing_cols = [col for col in cols_to_keep if col not in riv.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in riv DataFrame: {missing_cols}")
    riv = riv[cols_to_keep]

    # make sure the COMID and unitarea are defined in the cat
    cols_to_keep = ["COMID", "unitarea", "geometry"]
    missing_cols = [col for col in cols_to_keep  if col not in cat.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in cat DataFrame: {missing_cols}")
    cat = cat[cols_to_keep]
    
    # save the shapefile and network topology
    cat.to_file(path_out_temp+'cat.gpkg')
    riv.to_file(path_out_temp+'riv.gpkg')

    # pass lakeO to lake with copy
    lake = copy.deepcopy(lakeO)
    min_lon, min_lat, max_lon, max_lat = cat.total_bounds
    min_lon, min_lat, max_lon, max_lat = min_lon-2, min_lat-2, max_lon+2, max_lat+2 # add two degree buffer
    # subset lake
    lake_subset = lake
    lake_subset = lake_subset[lake_subset['x']<max_lon]
    lake_subset = lake_subset[lake_subset['x']>min_lon]
    lake_subset = lake_subset[lake_subset['y']<max_lat]
    lake_subset = lake_subset[lake_subset['y']>min_lat]

    # remove the lakes that are not intersecting with the cat
    # Find all lakes that intersect with any catchment
    intersections = gpd.sjoin(lake_subset, cat, how="inner", predicate="intersects")
    # Extract unique lake IDs (replace 'lake_id' with your actual column)
    lake_ids = intersections['Hylak_id'].unique()
    # Subset original lakes
    lake_subset_intersected = lake_subset[lake_subset['Hylak_id'].isin(lake_ids)].reset_index(drop=True)
    # rename the values
    lake_subset_intersected ['LakeCOMID'] = lake_subset_intersected ['Hylak_id']
    lake_subset_intersected ['Lakearea'] = lake_subset_intersected ['Lake_area']
    cols_to_keep = ["LakeCOMID", "Lakearea", "geometry"]
    missing_cols = [col for col in cols_to_keep if col not in lake_subset_intersected.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in lake DataFrame: {missing_cols}")
    lake_subset_intersected = lake_subset_intersected[cols_to_keep]
    # Save
    lake_subset_intersected.to_file(path_out_temp+'lake.gpkg')
        

# Identify the Resolvabales Lakes and Reservoirs, Indetify the Issues that Should be Resolved Within `HydroLakesCorrection.py`

In [4]:
# load the river and catchment files
if IncludeLakes:
    
    for pfaf in pfafs:

        print(f"=== Processing PFAF {pfaf} Resolvable Lakes and Reservoirs ===")
    
        # read the pfaf
        riv = gpd.read_file(TempFolder+'pfaf'+pfaf+'/pfaf'+pfaf+'/riv.gpkg')
        cat = gpd.read_file(TempFolder+'pfaf'+pfaf+'/pfaf'+pfaf+'/cat.gpkg')
        lake = gpd.read_file(TempFolder+'pfaf'+pfaf+'/pfaf'+pfaf+'/lake.gpkg')
    
        # max the COMID as it includes the merit hillslope also
        # the lake will be assigned higher COMID the maxed COMID
        maxCOMID = cat['COMID'].max()

        # remove the lakes that are entirely within the subbasin of merit-basins
        cat_lake_int = gpd.overlay(cat, lake, how = 'intersection')
        # Count occurrences of 'Hylak_id' in the intersection
        Hylak_id_counts = cat_lake_int['LakeCOMID'].value_counts()
        # Identify Hylak_id that appear only once
        single_occurrence_Hylak_id = list(Hylak_id_counts[Hylak_id_counts == 1].index)
        # Remove those from lake
        lake = lake[~lake['LakeCOMID'].isin(single_occurrence_Hylak_id)].reset_index(drop=True)
        
        # intersection of river segment and lakes
        river_lake_int = gpd.overlay(riv, lake, how = 'intersection')
        # Number of unique lakes in the intersection
        num_lakes = len(river_lake_int['LakeCOMID'].unique())
        print('Number of lakes in the intersection:', num_lakes)
        # Count the number of river segments that intersect with more than one lake
        m = (river_lake_int.groupby('COMID')['LakeCOMID']
             .nunique()
             .gt(1)
             .sum())
        print('Number of river segments that intersect with more than one lake:', m)
    
        # Step 1: 
        # Remove the smaller lakes for river segments that have more than one lake 
        # intersecting with them in the river_intersection_lake
        # Identify the largest lake area for each river segment and flag others for removal
        # Using `groupby` to mark lakes with non-maximum areas for each segment
        river_lake_int['remove'] = river_lake_int['Lakearea'] != river_lake_int.groupby('COMID')['Lakearea'].transform('max')
        # Ensure that if any row with a specific Lake_ID has remove=True, all rows with that Lake_ID are also set to remove=True
        lake_ids_to_remove = river_lake_int.loc[river_lake_int['remove'], 'LakeCOMID'].unique()
        river_lake_int.loc[river_lake_int['LakeCOMID'].isin(lake_ids_to_remove), 'remove'] = True
        # Retain only lakes with the largest area per segment
        river_lake_int = river_lake_int[river_lake_int['remove'] == False].sort_values(by='COMID').reset_index(drop=True)
        # Display the count of unique lakes after filtering
        print('Number of lakes in the simplified intersection step-1:', len(river_lake_int['LakeCOMID'].unique()))
        
        # Step 2:
        # Remove the lakes that have only on river segment passing through them (unresolved lakes)
        # Identify lakes that intersect with more than one river segment
        river_lake_int['keep'] = river_lake_int.groupby('LakeCOMID')['COMID'].transform('nunique') > 1
        # Keep only the lakes that intersect with more than one river segment
        river_lake_int = river_lake_int[river_lake_int['keep']].sort_values(by='COMID').reset_index(drop=True)
        # Display the number of unique lakes in the simplified intersection
        print('Number of lakes in the simplified intersection step-2:', river_lake_int['LakeCOMID'].nunique())
    
        # check 1:
        segments_with_multiple_lakes = river_lake_int.groupby('COMID')['LakeCOMID'].nunique()
        multiple_lake_segments = segments_with_multiple_lakes[segments_with_multiple_lakes > 1]
        # Display details for segments that intersect with multiple lakes
        for COMID, lake_count in multiple_lake_segments.items():
            lake_ids = river_lake_int.loc[river_lake_int['COMID'] == COMID, 'LakeCOMID'].unique()
            print(f"Segment ID: {COMID}")
            print("Number of lakes:", lake_count)
            print("Lake IDs:", lake_ids)
            print("----")
        # Summary check
        m = len(multiple_lake_segments)
        print('Number of river segments that intersect with more than one lake:', m)
        print("Good to continue" if m == 0 else "Something is wrong! Check the lake-river intersections.")
    
        # check 2:
        # Check for segments where `lake_id` count does not match the row count (indicating duplicates)
        duplicates_exist = (river_lake_int.groupby('COMID')['LakeCOMID'].nunique() != river_lake_int.groupby('COMID').size()).any()
        # Reporting
        if duplicates_exist:
            print("It seems there are less seg_id than elements of the river and lake intersection;")
            print("River and lake intersection should be dissolved on seg_id and lake_id.")
        else:
            print("It seems there is one seg_id for each element of the river and lake intersection.")
            print("Good to go.")

        # create error
        if duplicates_exist and m > 0:
            raise ValueError(f"Duplicate records detected: {m} duplicates found. Please resolve them before proceeding.")
    
        # slice the lake based on the Hylake_id
        resolvabale_lakes = lake[lake['LakeCOMID'].isin(river_lake_int['LakeCOMID'])].reset_index()
        resolvabale_lakes = resolvabale_lakes.sort_values(by="LakeCOMID").reset_index(drop=True)
        resolvabale_lakes ['COMID'] = np.arange(maxCOMID+1, maxCOMID+len(resolvabale_lakes)+1)
    
        # save resolvabale lakes
        path_out_temp = path_out+'pfaf'+pfaf+'/'+'pfaf'+pfaf+'/'
        if os.path.isdir(path_out_temp):
            try:
                shutil.rmtree(path_out_temp)  # Remove the entire directory and its contents
            except OSError as e:
                print(f"Error: {path_out_temp} - {e}")
        if not os.path.isdir(path_out_temp):
            os.makedirs(path_out_temp)
        resolvabale_lakes.to_file(path_out+'pfaf'+pfaf+'/pfaf'+pfaf+'/resolvable_lakes.gpkg')

=== Processing PFAF 72 Resolvable Lakes and Reservoirs ===
Number of lakes in the intersection: 7112
Number of river segments that intersect with more than one lake: 4106
Number of lakes in the simplified intersection step-1: 3713
Number of lakes in the simplified intersection step-2: 3386
Number of river segments that intersect with more than one lake: 0
Good to continue
It seems there is one seg_id for each element of the river and lake intersection.
Good to go.


# Burning the Lakes and Reservoirs Into the River Network.
## This Includes the following:
 - Correction of Subbasin Areas that are Fully or Partially Under Lakes
 - Correction of River Lengths that are Fully or Partially Under Lakes
 - Correction of Upstream and Downstream IDs for Both Lakes and Segments that Drain to or from Lakes or Reservoirs

In [5]:
# load the river and catchment files
for pfaf in pfafs:

    print(f"=== Processing PFAF {pfaf} Correcting Subbasins, River Segments, and Network Topology ===")

    if IncludeLakes:
    
        # load the riv, cat and resolvable lakes
        riv  = gpd.read_file(TempFolder+'pfaf'+pfaf+'/pfaf'+pfaf+'/riv.gpkg')
        cat  = gpd.read_file(TempFolder+'pfaf'+pfaf+'/pfaf'+pfaf+'/cat.gpkg')
        lake = gpd.read_file(path_out+'pfaf'+pfaf+'/pfaf'+pfaf+'/resolvable_lakes.gpkg')

        # create the ups
        cols_to_drop = [col for col in riv.columns
            if col in ['maxup'] or re.fullmatch(r"up\d+", col)]
        riv.drop(columns=cols_to_drop, inplace=True)
        riv =  add_immediate_upstream (riv, mapping = {'id':'COMID','next_id':'NextDownCOMID'})

        # create original graph from riv network topology
        original_graph = create_graph(riv['COMID'].tolist(), riv['NextDownCOMID'].tolist())
        
        # intersect the lake and riv
        riv_int = gpd.overlay(riv, lake, how = 'intersection')
        
        # carve out the lakes from cat
        cat_int = gpd.overlay(cat, lake, how = 'difference')
        
        # get the length of the riv, cat, riv_int, cat_int
        riv['length_org'] = riv.geometry.length
        cat['area_org'] = cat.geometry.area
        riv_int['length_in_lake'] = riv_int.geometry.length
        cat_int['area_out_lake'] = cat_int.geometry.area
    
        ####
        # Suppress all warnings
        warnings.filterwarnings("ignore")
        
        # step 1 - 
        # loop over the resolvable lakes, get the river segment that intersect with the lakes
        # for the segments with 1 to n-1 segment set the downstream as lake and flag inflow as 1 (outflow as zero)
        # for the segment n with larger uparea set the flag outflow as 1 (inflow as zero)
        # Initialize inflow and outflow with default values
        riv_int['inflow'] = False
        riv_int['outflow'] = False
        riv_int['endorheic'] = 0
        riv_int['exorheic'] = 0
        # Iterate over groups of Hylak_id
        for hylak_id, group in riv_int.groupby('LakeCOMID'):
            # Check the number of networks
            num_parts, components = count_network_parts(original_graph, group['COMID_1'].tolist())
            if num_parts == 1:
                # If there's only one network, assign outflow and inflow based on uparea
                riv_int.loc[group.index, 'outflow'] = group['uparea'] == group['uparea'].max()
                riv_int.loc[group.index, 'inflow'] = group['uparea'] != group['uparea'].max()
                riv_int.loc[group.index, 'exorheic'] = 1
            else:
                # If multiple networks, assign inflow to all segments in the Hylak_id group
                riv_int.loc[group.index, 'inflow'] = True
                riv_int.loc[group.index, 'endorheic'] = 1
        #riv_int['outflow'] = riv_int.groupby('Hylak_id')['uparea'].transform('max') == riv_int['uparea']
        #riv_int['inflow']  = riv_int.groupby('Hylak_id')['uparea'].transform('max') != riv_int['uparea']
        #print(riv_int)
        
        # step 2 - provide the length ratio of river segments;
        # for example length ratio of 0 that is in the intersect identify the entire
        # river is under the lake values
        # Initialize 'length_ratio' column with 1.0
        riv['length_ratio'] = 1.0
        # Map the 'length_in_lake' from riv_int to riv based on matching 'COMID'
        length_in_lake_map = riv_int.set_index('COMID_1')['length_in_lake']
        # Update 'length_ratio' for rows where 'COMID' exists in riv_int
        riv['length_in_lake'] = riv['COMID'].map(length_in_lake_map).fillna(0)
        riv['length_ratio'] = 1 - (riv['length_in_lake'] / riv['length_org'])
        
        # step 3 - update the area ratio
        # Initialize 'area_ratio' column with 1.0 in cat
        cat['area_ratio'] = 1.0
        # Map the 'area_out_lake' from cat_int to cat based on matching 'COMID_1'
        area_out_lake_map = cat_int.set_index('COMID')['area_out_lake']
        # Update 'area_out_lake' in cat for rows where 'COMID' exists in cat_int
        cat['area_out_lake'] = cat['COMID'].map(area_out_lake_map).fillna(0)
        # Update the 'area' in cat based on the ratio of 'area_out_lake' to 'area_org'
        cat['area_ratio'] = cat['area_out_lake'] / cat['area_org']
        # Map 'area_ratio', 'area_out_lake', and 'area_org' from cat to riv based on COMID
        riv = riv.merge(cat[['COMID', 'area_ratio', 'area_out_lake', 'area_org']], on='COMID', how='left')
        
        # step 4 - for the segments that are identified as inflow for the lake;
        # turn the downstream to the COMID of that lake
        # Initial the LakeInflow Flag
        riv['LakeInflow'] = 0
        # Filter riv_int where 'inflow' is set to 1
        filtered_riv_int = riv_int[riv_int['inflow'] == 1]
        # Create a mapping of 'COMID_1' to 'COMID_2' from the filtered riv_int
        comid_mapping = filtered_riv_int.set_index('COMID_1')['COMID_2']
        # Update the 'DownNextID' in riv only for rows where 'COMID' exists in the mapping
        riv.loc[riv['COMID'].isin(comid_mapping.index), 'NextDownCOMID'] = riv['COMID'].map(comid_mapping)
        riv.loc[riv['COMID'].isin(comid_mapping.index), 'LakeInflow'] = 1
        
        # step 5 - for the segment that area identified as outflow from the lake;
        # trun the upstream to that lake ID as well
        # Filter riv_int where 'outflow' is set to 1
        filtered_riv_int = riv_int[riv_int['outflow'] == 1]
        # Create a mapping of 'COMID_1' to 'LakeOutflow' from the filtered riv_int
        lake_outflow_mapping = filtered_riv_int.set_index('COMID_1')['outflow']
        # Update the 'LakeOutflow' column in riv based on the mapping
        riv['LakeOutflow'] = 0  # Initialize the column with 0
        riv.loc[riv['COMID'].isin(lake_outflow_mapping.index), 'LakeOutflow'] = \
            riv['COMID'].map(lake_outflow_mapping).fillna(0).astype(int)
        
        # step 6 - add COMID_2 from the lake and update the riv
        # Select the necessary columns from riv_int for the new rows
        new_rows = riv_int[['COMID_2', 'LakeCOMID', 'endorheic', 'exorheic']].copy()
        # Rename COMID_2 to match the COMID column in riv
        new_rows.rename(columns={'COMID_2': 'COMID'}, inplace=True)
        # Remove duplicates based on 'COMID_2' and 'Hylak_id', keeping the first occurrence
        new_rows = new_rows.drop_duplicates(subset=['COMID', 'LakeCOMID', 'endorheic', 'exorheic'], keep='first')
        # Add the 'islake' column and set it to 1 for the new rows
        new_rows['islake'] = 1
        # Ensure all columns in new_rows align with riv's structure
        # Add missing columns from riv to new_rows with default values (e.g., NaN)
        for col in riv.columns:
            if col not in new_rows.columns:
                new_rows[col] = None  # Default values for missing columns
        ## Ensure any extra columns in new_rows are removed
        #new_rows = new_rows[riv.columns]
        # Append new_rows to riv
        riv = pd.concat([riv, new_rows], ignore_index=True)
        
        # step 7 - for COMID that are lake in riv, islake is 1, find the riv_int outflow from the riv_int
        # add that COMID_1 from them riv_int that outflow is true to the NextDownCOMID
        # this is to update the downstream of the lake riv
        # Remove rows from riv_int where 'outflow' is False
        riv_int_filtered = riv_int[riv_int['outflow'] == True].copy()
        # Ensure COMID_1 and COMID_2 pairs are unique
        if riv_int_filtered.duplicated(subset=['COMID_1', 'COMID_2']).any():
            raise ValueError("Duplicate COMID_1 and COMID_2 pairs found in riv_int!")
        # Update NextDownCOMID in riv
        # Merge riv with riv_int_filtered on COMID_2
        riv = riv.merge(
            riv_int_filtered[['COMID_1', 'COMID_2']], 
            left_on='COMID', 
            right_on='COMID_2', 
            how='left')
        # Replace NextDownCOMID in riv with COMID_1 from riv_int_filtered where there's a match
        riv['NextDownCOMID'] = riv['COMID_1'].combine_first(riv['NextDownCOMID'])
        # Drop temporary merge columns
        riv.drop(columns=['COMID_1', 'COMID_2'], inplace=True)
        
        # step 8 - update the upstreams
        riv['maxup_org'] = 0
        riv['diffmaxup'] = 0
        riv['maxup_org'] = riv['maxup']
        riv['NextDownCOMID'] = riv['NextDownCOMID'].fillna(-1).astype(int)
        # Drop columns named exactly in explicit_drop OR matching pattern up<digit>
        cols_to_drop = [col for col in riv.columns
            if col in ['maxup'] or re.fullmatch(r"up\d+", col)]
        riv.drop(columns=cols_to_drop, inplace=True)
        # generate the ups again for riv and lake
        riv =  add_immediate_upstream (riv, mapping = {'id':'COMID','next_id':'NextDownCOMID'})
        riv['diffmaxup'] = riv['maxup_org'] - riv['maxup']
        riv['diffmaxup'] = riv['diffmaxup'].abs()
    
        # step 9A - correction of area and length of river network values
        for index, row in riv.iterrows():
            # check if the cat and riv length ratio are changed; intersection with lakes
            if row['area_ratio'] < 1 or row['length_ratio'] < 1:
                if row['LakeInflow'] == 1: # the the segments are identified as inflow for the lake
                    if 0 <= row['area_ratio'] and 0 <= row['length_ratio']: # check if both of the area and length ratio are above 0
                        # meaning both cat and river are existing
                        # condition 1,  max up is not changed
                        if row['diffmaxup'] == 0:
                            # the order is not changed
                            riv.loc[index, 'uparea'] = riv.loc[index, 'uparea'] - \
                                                       riv.loc[index, 'unitarea'] * (1-riv.loc[index, 'area_ratio'])# updating uparea
                            riv.loc[index, 'unitarea'] = riv.loc[index, 'unitarea'] * riv.loc[index, 'area_ratio'] # updating unitarea
                            riv.loc[index, 'length'] = riv.loc[index, 'length'] * riv.loc[index, 'length_ratio'] # updating length
                        else: # the uparea is changed
                            # condition 2, maxup is 0 then uparea is set to corrected unitarea
                            if row['maxup'] == 0:
                                riv.loc[index, 'uparea'] = riv.loc[index, 'unitarea'] * cat.loc[index,'area_ratio']
                                riv.loc[index, 'unitarea'] = riv.loc[index, 'unitarea'] * riv.loc[index, 'area_ratio']
                                riv.loc[index, 'length'] = riv.loc[index, 'length'] * riv.loc[index, 'length_ratio']
                                # riv.loc[index, 'length'] = 0.0
                elif row['LakeOutflow'] == 1: # it should be outflow then
                    if 0 <= row['area_ratio'] and 0 <= row['length_ratio']: # check if both of the area and length ratio are above 0
                        riv.loc[index, 'unitarea'] = riv.loc[index, 'unitarea'] * riv.loc[index, 'area_ratio']
                        riv.loc[index, 'length'] = riv.loc[index, 'length'] * riv.loc[index, 'length_ratio']
                        # uparea remains unchanged 
    
        # step 9B - correction of unit area and uparea of the lake objects
        # check if it is a lake and update the uparea and also the unitarea
        up_cols = [col for col in riv.columns if re.match(r'^up\d+$', col)]
        for index, row in riv.iterrows():
            if row['islake'] == 1:
                # unit area
                idx = lake.loc[lake['COMID'] == row['COMID']].index
                riv.loc[index, 'unitarea'] = lake.loc[idx[0], 'Lakearea']
                # uparea: get the existing id of the up, and sum the values from the riv for those + the unit area
                # get the upstream COMIDs
                up_COMIDs = list(set(row[up_cols]))
                riv_slice = riv[riv['COMID'].isin(up_COMIDs)]
                riv.loc[index, 'uparea'] = riv.loc[index, 'unitarea'] # initialize the lake up area
                if not riv_slice.empty:
                    riv.loc[index, 'uparea'] = riv_slice['uparea'].sum()+riv.loc[index, 'uparea'] # upstream area + unit area of lake
    
        # step 10 - move the next down ID for endorheic basin to -9999
        for index, row in riv.iterrows():
            if row['endorheic'] == 1:
                riv.loc[index, 'NextDownCOMID'] = -9999
        
        # step 11 - remove the COMIDs that both the riv and cat are fully under the lakes
        riv['remove_riv'] = 0
        riv['remove_cat'] = 0
        riv['remove_riv_cat'] = 0
        for index, row in riv.iterrows():
            if row['length_ratio'] == 0:
                riv.loc[index, 'remove_riv'] = 1
            if row['area_ratio'] == 0:
                riv.loc[index, 'remove_cat'] = 1
            if row['length_ratio'] == 0 and row['area_ratio'] == 0:
                riv.loc[index, 'remove_riv_cat'] = 1
    
        # step 12 - replace the geometry of riv
        # carve out the lakes from the river segments
        riv_carved = gpd.overlay(riv, lake, how = 'difference')
        # replace the geometry from resolvable lakes or riv_carved looping in the riv
        for index, row in riv.iterrows():
            if row['length_ratio'] < 1: # replace the geometry with corrected riv
                # find the idx of the riv_carved based on the COMID
                idx = riv_carved.loc[riv_carved['COMID'] == row['COMID']].index
                if len(idx) == 1:
                    riv.loc[index, 'geometry'] = riv_carved.loc[idx[0], 'geometry']
                    #print(f"Expected one matching index for COMID {row['COMID']}, but got {len(idx)}.")
                if len(idx) > 1:
                    raise ValueError(f"Expected one matching index for COMID {row['COMID']}, but got {len(idx)}.")
            if row['remove_riv'] == 1: # remove the river fully
                riv.loc[index, 'geometry'] = None
            if row['islake'] == 1: # is lake, add lake to riv_lake
                # find the idx of the lake based on the COMID
                idx = lake.loc[lake['COMID'] == row['COMID']].index
                #print(idx)
                if len(idx) != 1:
                    raise ValueError(f"Expected one matching index for COMID {row['COMID']}, but got {len(idx)}.")
                riv.loc[index, 'geometry'] = lake.loc[idx[0], 'geometry']
        # clean up
        #riv = riv.rename(columns={'hillslope': 'merit_hillslope'})
        # riv['hillslope'] = riv['hillslope'].apply(lambda x: 1 if x == 1 else 0)
        # riv = riv.drop(columns = ['width', 'submodel', 'submodel_order', 'station_id', 'length_org', \
        #                           'length_ratio', 'length_in_lake', 'area_ratio', 'area_out_lake', \
        #                           'area_org', 'maxup_org', 'diffmaxup', 'order'])
    
        # step 13 - save the geometry of the corrected cat
        cat = pd.concat([lake[['COMID', 'geometry']], cat_int[['COMID', 'geometry']]], ignore_index=True)
        # Merge the two DataFrames based on the 'COMID' key
        cat = cat.merge(riv[['COMID'] + ['LakeCOMID', 'endorheic', 'exorheic', 'islake', 'unitarea']],\
                        on='COMID', how='left')
        # clean up and assign the area from hydrolakes
        #cat = cat.rename(columns={'hillslope': 'merit_hillslope'})
        for index, row in cat.iterrows():
            if row['islake'] ==1 : # replace the area from the lake 
                idx = lake.loc[lake['COMID'] == row['COMID']].index
                if len(idx) == 1:
                    cat.loc[index, 'geometry'] = lake.loc[idx[0], 'geometry']
    
        # step 14 - clean up and redo the up to remove the up identified COMID that are removed
        riv = riv[riv['remove_riv_cat'] != 1].sort_values(by='COMID', ascending=True).reset_index(drop=True)
        riv = riv.drop(riv.filter(regex=r'^up\d+$').columns, axis=1)
        riv = add_immediate_upstream (riv, mapping = {'id':'COMID','next_id':'NextDownCOMID'})
        cat = cat.sort_values(by='COMID', ascending=True).reset_index(drop=True)
    
        # Step 15 - Pass the columns from the hydrolakes to riv with columns as hydrolake_ (excluding geometry)
        for index, row in riv.iterrows():
            if row['islake'] == 1:
                # Find the index of the matching row in the lake DataFrame
                idx = lake.loc[lake['COMID'] == row['COMID']].index
                # Ensure the index is not empty (there's a match)
                if not idx.empty:
                    lake_row = lake.loc[idx[0]]  # Get the first matching row as a Series
                    # Add the lake data to the corresponding row in riv with 'hydrolake_' prefix, excluding 'geometry'
                    for col in lake.columns:
                        if col != 'geometry':  # Exclude the 'geometry' column
                            riv.loc[index, f'from_lake_{col}'] = lake_row[col]
    
        # Step 16 - length to zero for lakes
        for index, row in riv.iterrows():
            if row['islake'] == 1:
                # update the values
                riv.loc[index, 'length'] = 0.00

        # Step 17 - coastal subbasins, the length is zero but subbasin exists
        riv['coastal'] = 0
        for index, row in riv.iterrows():
            if row['islake'] != 1 and row['length'] == 0:
                # update the values
                riv.loc[index, 'coastal'] = 1
        cat ['coastal'] = riv['coastal']

        # Step - 18 clean up the the existing column name if present
        col_to_remove = ["length_org", "length_ratio", "length_in_lake", \
                         "area_ratio", "area_out_lake", "area_org", "maxup_org", \
                         "diffmaxup", "remove_riv", "remove_cat", "remove_riv_cat"]
        # Keep only columns that are actually present in riv
        cols_existing = [c for c in col_to_remove if c in riv.columns]
        # Drop them
        riv.drop(columns=cols_existing, inplace=True)

        # Step - 19 save the riv and cat
        # Save the shapefiles
        riv.to_file(path_out+'pfaf'+pfaf+'/pfaf'+pfaf+'/riv.gpkg')
        cat.to_file(path_out+'pfaf'+pfaf+'/pfaf'+pfaf+'/cat.gpkg')
    
        # Re-enable warnings
        warnings.filterwarnings("default")

    else:

        if not os.path.isdir(path_out+'pfaf'+pfaf+'/pfaf'+pfaf):
            os.makedirs(path_out+'pfaf'+pfaf+'/pfaf'+pfaf)
        # no lake, move riv and cat to the path_out from Temp folder
        shutil.move(TempFolder+'pfaf'+pfaf+'/pfaf'+pfaf+'/riv.gpkg',\
                    path_out+'pfaf'+pfaf+'/pfaf'+pfaf+'/riv.gpkg')
        shutil.move(TempFolder+'pfaf'+pfaf+'/pfaf'+pfaf+'/cat.gpkg', \
                    path_out+'pfaf'+pfaf+'/pfaf'+pfaf+'/cat.gpkg')

=== Processing PFAF 72 Correcting Subbasins, River Segments, and Network Topology ===


In [6]:
warnings.resetwarnings()